In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

# Datos de ejemplo

In [3]:
import pandas as pd
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach'],
        'x3': [1, 1, 2, 2, 2, 4],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5],
        'y1': [1, 0, 1, 0, 0, 1],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes']
    })
df = spark.createDataFrame(pdf)
df.show()

+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
+---+------+---+---+---+---+



# StringIndexer

`StringIndexer` asigna una columna de cadena a una columna de índice que será tratada como una columna categórica por chispa. **Los índices comienzan con 0 y están ordenados por frecuencias de etiqueta**. Si se trata de una columna numérica, la columna se lanzará primero a una columna de cadena y luego se indexará mediante StringIndexer.

Hay tres pasos para implementar el StringIndexer

1. Construir el modelo StringIndexer: especifique los nombres de las columnas de entrada y salida.
2. Aprenda el modelo StringIndexer: ajuste el modelo con sus datos.
3. Ejecutar la indexación: llamar a la función transformar para ejecutar el proceso de indexación.

### Ejemplo: `StringIndex` columna "x1"

In [4]:
from pyspark.ml.feature import StringIndexer

# build indexer
string_indexer = StringIndexer(inputCol='x1', outputCol='indexed_x1')

# learn the model
string_indexer_model = string_indexer.fit(df)

# transform the data
df_stringindexer = string_indexer_model.transform(df)

# resulting df
df_stringindexer.show()

+---+------+---+---+---+---+----------+
| x1|    x2| x3| x4| y1| y2|indexed_x1|
+---+------+---+---+---+---+----------+
|  a| apple|  1|2.4|  1|yes|       1.0|
|  a|orange|  1|2.5|  0| no|       1.0|
|  b|orange|  2|3.5|  1| no|       0.0|
|  b|orange|  2|1.4|  0|yes|       0.0|
|  b| peach|  2|2.1|  0|yes|       0.0|
|  c| peach|  4|1.5|  1|yes|       2.0|
+---+------+---+---+---+---+----------+



Del resultado anterior, podemos ver que (a, b, c) en la columna x1 se convierten a (1.0, 0.0, 2.0). Están ordenados por sus frecuencias en la columna x1.


# OneHotEncoder

**`OneHotEncoder`** convierte cada categoría de una columna **StringIndexed** en un `vector espeso`. Cada vector disperso tiene **como máximo un solo elemento activo** que indica el índice de la categoría.

In [7]:
df_ohe = df.select('x1')
df_ohe.show()

+---+
| x1|
+---+
|  a|
|  a|
|  b|
|  b|
|  b|
|  c|
+---+



### `StringIndex` columna 'x1'

In [14]:
df_x1_indexed = StringIndexer(inputCol='x1', outputCol='indexed_x1').fit(df_ohe).transform(df_ohe)
df_x1_indexed.show()

+---+----------+
| x1|indexed_x1|
+---+----------+
|  a|       1.0|
|  a|       1.0|
|  b|       0.0|
|  b|       0.0|
|  b|       0.0|
|  c|       2.0|
+---+----------+



x1" tiene tres categorías: "a", "b" y "c", que corresponden a los índices de cadenas 1,0, 0,0 y 2,0, respectivamente.

### Asignación de índices de cadenas a vectores dispersos

* Formato de codificación: índice de cadenas": index of string indices vector size", "index of string index in string indices vector", **1.0** ]

Aquí el vector de los índices de cadenas es `[0.0, 1.0, 2.0]`. Por lo tanto, el mapeo entre índices de cadenas y vectores dispersos es:
* `0.0: [3, [0], [1.0]]`
* `1.0: [3, [1], [1.0]]`
* `2.0: [3, [2], [1.0]]`

Después de convertir todos los vectores dispersos en vectores densos, obtenemos:

In [43]:
from pyspark.ml.linalg import DenseVector, SparseVector, DenseMatrix, SparseMatrix
x = [SparseVector(3, {0: 1.0}).toArray()] + \
    [SparseVector(3, {1: 1.0}).toArray()] + \
    [SparseVector(3, {2: 1.0}).toArray()]

import numpy as np
np.array(x)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

**La matriz obtenida es exactamente la matriz que utilizaríamos para representar nuestra variable categórica en una clase estadística**.

### Un paso más para dar

`OneHotEncoder` de forma predeterminada dejará caer la última categoría. Así que el **vector de índices de cadenas** se convierte en `[0.0, 1.0]`, y los mapeos entre índices de cadenas y vectores dispersos lo son:

* `0.0: [2, [0], [1.0]]`
* `1.0: [2, [1], [1.0]]`
* `2.0: [2, [], []]`

Utilizamos un vector disperso que tiene **ningún elemento activo** (básicamente todos los elementos son 0's) para representar la última categoría.

# Verificar

### OneHotEncode columna 'indexed_x1'

In [46]:
from pyspark.ml.feature import OneHotEncoder

In [47]:
OneHotEncoder(inputCol='indexed_x1', outputCol='encoded_x1').transform(df_x1_indexed).show()

+---+----------+-------------+
| x1|indexed_x1|   encoded_x1|
+---+----------+-------------+
|  a|       1.0|(2,[1],[1.0])|
|  a|       1.0|(2,[1],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  c|       2.0|    (2,[],[])|
+---+----------+-------------+



### Especifique no dejar caer la última categoría

Si decidimos no abandonar la última categoría, obtenemos los resultados esperados.

In [49]:
OneHotEncoder(dropLast=False, inputCol='indexed_x1', outputCol='encoded_x1').transform(df_x1_indexed).show()

+---+----------+-------------+
| x1|indexed_x1|   encoded_x1|
+---+----------+-------------+
|  a|       1.0|(3,[1],[1.0])|
|  a|       1.0|(3,[1],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  c|       2.0|(3,[2],[1.0])|
+---+----------+-------------+

